## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-11-02-00 +0000,wsj,Auto & Transport Roundup: Market Talk,https://www.wsj.com/business/auto-transport-ro...
1,2026-02-20-11-01-00 +0000,wsj,Moncler’s Shares Surge After Upbeat End to the...,https://www.wsj.com/business/retail/monclers-s...
2,2026-02-20-11-00-00 +0000,nypost,Why Nancy Guthrie hasn’t been found — from cop...,https://nypost.com/2026/02/20/us-news/why-nanc...
3,2026-02-20-11-00-00 +0000,nypost,Eagle-eyed cops nab NYC transit terrors in jus...,https://nypost.com/2026/02/20/us-news/eagle-ey...
4,2026-02-20-11-00-00 +0000,wapo,Majority of Americans think Trump’s deportatio...,https://www.washingtonpost.com/immigration/202...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2479/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,58
351,andrew,29
156,epstein,28
350,prince,25
344,iran,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,176
49,2026-02-20-08-09-06 +0000,nyt,British Police Arrest Former Prince Andrew Ami...,https://www.nytimes.com/2026/02/19/world/europ...,157
186,2026-02-19-22-28-26 +0000,nypost,Bombshell Jeffrey Epstein files that led to ex...,https://nypost.com/2026/02/19/world-news/these...,148
231,2026-02-19-20-28-03 +0000,nyt,The police are investigating former Prince And...,https://www.nytimes.com/live/2026/02/19/world/...,142
251,2026-02-19-19-31-44 +0000,nypost,Ex-Prince Andrew looks aghast in first photos ...,https://nypost.com/2026/02/19/world-news/ex-pr...,141


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,176,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...
49,75,2026-02-20-08-09-06 +0000,nyt,British Police Arrest Former Prince Andrew Ami...,https://www.nytimes.com/2026/02/19/world/europ...
318,61,2026-02-19-15-39-00 +0000,wsj,"The U.S. trade deficit jumped in December, the...",https://www.wsj.com/economy/trade/u-s-trade-de...
259,58,2026-02-19-19-09-03 +0000,nypost,UK blocks Trump from using RAF bases for strik...,https://nypost.com/2026/02/19/world-news/uk-bl...
77,56,2026-02-20-04-44-31 +0000,nypost,New Tahoe ski death as student found on black ...,https://nypost.com/2026/02/19/us-news/new-taho...
150,52,2026-02-20-00-00-00 +0000,wsj,The husband of embattled Labor Secretary Lori ...,https://www.wsj.com/politics/policy/labor-secr...
88,48,2026-02-20-03-11-44 +0000,nypost,Texas woman missing after failing to report to...,https://nypost.com/2026/02/19/us-news/texas-wo...
96,47,2026-02-20-02-41-00 +0000,wsj,President Trump intends to order the release o...,https://www.wsj.com/politics/national-security...
349,44,2026-02-19-13-48-20 +0000,nypost,Middle Eastern funds are ready to invest tens ...,https://nypost.com/2026/02/19/business/jpmorga...
295,38,2026-02-19-17-07-21 +0000,nypost,Chaos at major AI summit as scandal-plagued Bi...,https://nypost.com/2026/02/19/business/bill-ga...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
